## NLP Setup

In [1]:
import random
import operator
import pickle
import sys
import spacy
from spacy import displacy
import benepar
import copy
import networkx as nx

In [2]:
# Set up and store NLP
nlp = spacy.load("en_core_web_md")
benepar.download('benepar_en3')
nlp.add_pipe('benepar', config={'model': 'benepar_en3'})

with open('./nlp/nlp.pickle', 'wb') as f:
    pickle.dump(nlp, f)

[nltk_data] Downloading package benepar_en3 to
[nltk_data]     C:\Users\regan\AppData\Roaming\nltk_data...
[nltk_data]   Package benepar_en3 is already up-to-date!


In [3]:
# Load NLP
#sys.path.append('../')

with open('./nlp/nlp.pickle', 'rb') as f:
    nlp = pickle.load(f)

## Graph

In [4]:
from app.classes.spec.symboleo_spec import PAtom
from app.src.graph.graph_builder import GraphBuilder
from app.src.graph.graph_visualizer import GraphVisualizer

graph_builder = GraphBuilder()
graph_visualizer = GraphVisualizer()

digraph = graph_builder.build(PAtom)

gv = graph_visualizer.create_viz(digraph.nodes)
gv.show('nx.html')


## Set up Extractor

In [ ]:
from app.src.atom_assembler import AtomAssembler
from app.src.atom_extractor import AtomExtractor
from app.src.predicate_builder import PredicateBuilder
from app.src.primitive_checker import PrimitiveChecker
from app.src.primitive_extractor import PrimitiveExtractor
from app.src.dynamic_constructor import DynamicObjectConstructor
from app.src.sentence_summarizer import SentenceSummarizer

summarizer = SentenceSummarizer(nlp)

options = {}

primitive_identifiers = [] # Will need to add this in
primitive_extractor = PrimitiveExtractor(primitive_identifiers)

candidates = [
    'PredicateFunctionHappens',
    'PredicateFunctionHappensWithin',
    'PredicateFunctionOccurs',
    'PredicateFunctionWHappensBefore',
    'PredicateFunctionWHappensBeforeEvent',
    'PredicateFunctionHappensAfter',
]

primitive_checker = PrimitiveChecker()
dynamic_constructor = DynamicObjectConstructor(digraph)
predicate_builder = PredicateBuilder(digraph, primitive_checker, dynamic_constructor)
atom_assembler = AtomAssembler(candidates, predicate_builder)

atom_extractor = AtomExtractor(options, nlp, primitive_extractor, atom_assembler)

In [ ]:
sentence = ''

# summarize the sentence
summarizer.summarize(sentence)

results = atom_extractor.extract(sentence)

for result in results:
    print(result.to_sym())

## Gather user input (old)

In [ ]:
from app.classes.spec.symboleo_spec import Obligation
from app.templates.meat_sale.norms import meat_sale_norms
from app.templates.meat_sale.templates import meat_sale_templates
obligations: list[Obligation] = copy.deepcopy(meat_sale_norms['obligations'])
templates = meat_sale_templates['obligations']

In [ ]:
USER_INPUT = True

DEFAULT_OBLIGATION_ID = 'delivery'
DEFAULT_COMPONENT_CHAR = 't'

COMPONENT_DICT = {
    'a': 'antecedent',
    't': 'trigger',
    'c': 'consequent'
}

SAMPLE_SENTENCES = [
    "It is a holiday",
    "The sky is blue",
    "It is not a holiday",
    "It is a full moon",
    "two plus two equals four",
    "the house is on fire",
    "Ted built a house",
    "it is expected to rain",
    "it is raining"
]

DEFAULT_SENTENCE = SAMPLE_SENTENCES[1]


In [ ]:
for ob in obligations:
    print(ob.id)
    print(f'- NL: {templates[ob.id]}')
    print(f'- SYM: {ob.to_sym()}')

In [ ]:
# Choose the obligation
if USER_INPUT:
    # Choose the obligation
    selected_ob_id = input('Enter obligation id: ')
else:
    selected_ob_id = DEFAULT_OBLIGATION_ID

In [ ]:
if USER_INPUT:
    selected_component_char = input('Select the component to update (t/a/c): ')
else:
    selected_component_char = DEFAULT_COMPONENT_CHAR

selected_component = COMPONENT_DICT[selected_component_char]

In [ ]:
if USER_INPUT:
    sentence = input('Enter an extension sentence: ')
else:
    sentence = DEFAULT_SENTENCE